# Instalación de librerías

In [ ]:
!pip install pycuda
!pip install biopython
!pip install numpy
!pip install matplotlib
!pip install tqdm
!pip install scipy


# Generación de Dotplot de comparación de secuencias

In [ ]:
import argparse
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import time

def read_fasta(file_path, max_length=None):
    """Lee una secuencia de un archivo FASTA y devuelve la secuencia."""
    start_time = time.time()
    for record in SeqIO.parse(file_path, "fasta"):
        sequence = str(record.seq)
        if max_length:
            sequence = sequence[:max_length]
        end_time = time.time()
        print(f"Tiempo de carga de datos para {file_path}: {end_time - start_time:.2f} segundos")
        return sequence

def encode_sequence(seq):
    """Codifica una secuencia de ADN en enteros."""
    encoding = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return np.array([encoding[nuc] for nuc in seq], dtype=np.uint8)

def generate_dotplot_cuda(seq1, seq2, window_size=3, block_size=16):
    """Genera una matriz dispersa de dotplot para dos secuencias utilizando PyCUDA."""
    len1, len2 = len(seq1), len(seq2)
    seq1_array = encode_sequence(seq1)
    seq2_array = encode_sequence(seq2)

    # Compilando el kernel CUDA
    mod = SourceModule("""
    __global__ void dotplot_kernel(unsigned char *seq1, unsigned char *seq2, int len1, int len2, int window_size, int *output) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        int idy = blockIdx.y * blockDim.y + threadIdx.y;

        if (idx < len1 - window_size + 1 && idy < len2 - window_size + 1) {
            bool match = true;
            for (int k = 0; k < window_size; k++) {
                if (seq1[idx + k] != seq2[idy + k]) {
                    match = false;
                    break;
                }
            }
            if (match) {
                output[(idx + window_size / 2) * len2 + (idy + window_size / 2)] = 1;
            }
        }
    }
    """)

    # Copiando las secuencias a la memoria de la GPU
    seq1_gpu = gpuarray.to_gpu(seq1_array)
    seq2_gpu = gpuarray.to_gpu(seq2_array)
    output_gpu = gpuarray.zeros((len1, len2), dtype=np.int32)

    # Definiendo el tamaño de los bloques y la grilla para el kernel CUDA
    block = (block_size, block_size, 1)
    grid = ((len1 + block[0] - 1) // block[0], (len2 + block[1] - 1) // block[1], 1)

    # Ejecutando el kernel CUDA
    kernel_start_time = time.time()
    dotplot_kernel = mod.get_function("dotplot_kernel")
    dotplot_kernel(seq1_gpu, seq2_gpu, np.int32(len1), np.int32(len2), np.int32(window_size), output_gpu, block=block, grid=grid)
    kernel_end_time = time.time()
    print(f"Tiempo de ejecución del kernel CUDA: {kernel_end_time - kernel_start_time:.2f} segundos")

    # Recuperando la matriz de coincidencias desde la GPU
    output = output_gpu.get()
    rows, cols = np.nonzero(output)
    dotplot = coo_matrix((np.ones(len(rows)), (rows, cols)), shape=(len1, len2), dtype=int)

    return dotplot.tocsr(), kernel_start_time, kernel_end_time

def plot_dotplot(dotplot, output_file):
    """Dibuja y guarda la imagen del dotplot."""
    start_time = time.time()
    plt.figure(figsize=(10, 10))
    plt.imshow(dotplot.toarray(), cmap='Greys', interpolation='none')
    plt.savefig(output_file, format='png')
    plt.close()
    end_time = time.time()
    print(f"Tiempo para generar y guardar la imagen: {end_time - start_time:.2f} segundos")

def main(file1, file2, output_file, max_length, block_size, window_size, sequential_time):
    start_time = time.time()  # Tiempo inicial para la ejecución del programa

    # Leyendo las secuencias desde los archivos FASTA
    seq1 = read_fasta(file1, max_length)
    seq2 = read_fasta(file2, max_length)

    print(f"Longitud de la secuencia 1: {len(seq1)}")
    print(f"Longitud de la secuencia 2: {len(seq2)}")

    # Calculando el dotplot usando CUDA
    calc_start_time = time.time()  # Tiempo inicial para los cálculos
    dotplot, kernel_start_time, kernel_end_time = generate_dotplot_cuda(seq1, seq2, window_size=window_size, block_size=block_size)
    calc_end_time = time.time()  # Tiempo final para los cálculos

    if dotplot is not None:
        print(f"Tiempo de cálculo para generar el dotplot: {calc_end_time - calc_start_time:.2f} segundos")
        # Dibujando y guardando el dotplot
        plot_dotplot(dotplot, output_file)
    else:
        print("No se pudo generar el dotplot debido a un error de memoria.")

    end_time = time.time()  # Tiempo final para la ejecución del programa
    total_execution_time = end_time - start_time
    print(f"Tiempo total de ejecución del programa: {total_execution_time:.2f} segundos")

    # Calculando métricas de rendimiento
    parallelizable_time = kernel_end_time - kernel_start_time
    non_parallelizable_time = total_execution_time - parallelizable_time
    speedup = sequential_time / total_execution_time
    efficiency = speedup / (parallelizable_time + non_parallelizable_time)

    print(f"Tiempo de ejecución total: {total_execution_time:.2f} segundos")
    print(f"Tiempo de carga de datos: {calc_start_time - start_time:.2f} segundos")
    print(f"Tiempo de generación de la imagen: {end_time - calc_end_time:.2f} segundos")
    print(f"Tiempo muerto (no empleado en la ejecución del problema): {non_parallelizable_time:.2f} segundos")
    print(f"Aceleración: {speedup:.2f}")
    print(f"Eficiencia: {efficiency:.2f}")

# Definir argumentos para el entorno Colab
class Args:
    file1 = '/content/sample_data/Salmonella.fna'
    file2 = '/content/sample_data/E_coli.fna'
    output = '/content/sample_data/dotplot_cuda.png'
    max_length = 10000
    window_size = 3
    block_size = 16

args = Args()
sequential_time = float(input("Introduce el tiempo secuencial (en segundos): "))
main(args.file1, args.file2, args.output, args.max_length, args.block_size, args.window_size, sequential_time)